# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

gmaps.configure(api_key=geoapify_key)


# Display the map
city_data_df = city_data_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) 
                                 & (city_data_df["Max Temp"] > 21) 
                                 & (city_data_df["Wind Speed"] < 4.5) 
                                 & (city_data_df["Cloudiness"] == 0),:].dropna()
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = filtered_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df = hotel_df.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])

hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE

hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# params dictionary to update each iteration
params = {
    "radius": 10000,
    "types": "hotel",
    "key": geoapify_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while keeping original params in place
    params["location"] = f"{lat},{lng}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display hotel names on dataframe
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name


In [7]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=21f4509cff2248d08c0daf0145006453"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)


200


In [8]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))